In [1]:
import numpy as np
from scipy import linalg

# Funçoes para a solução de sistemas lineares $Ax=b$

In [2]:
def lsolve(A, b):
    '''
    Resolução de sistemas lineares usando inversão de matrizes.
    
    Entrada:
        A - matriz de coeficientes
        b - vetor de termos independentes
    Saída:
        x - vetor de incógnitas.
    '''
    det_A = linalg.det(A)
    assert det_A != 0, 'O determinante de A é zero.:('
    A_inv = linalg.inv(A)
    x = A_inv @ b
    return x

def lsolve_triang_inf(A, b):
    '''
    Resolve sistemas lineares triangulares inferiores.
    
    Entrada:
        A - matriz de coeficientes
        b - vetor de termos independentes
    Saída:
        x - vetor de incógnitas.
    '''
    n = b.shape[0]
    x = np.empty(b.shape)
    x[0] = b[0] / A[0,0]
    for i in range(1,n):
        soma = 0
        for j in range(i):
            soma = soma + A[i,j]*x[j]
        x[i] = (b[i] - soma) / A[i,i]
    return x

def lsolve_triang_sup(A, b):
    '''
    Resolve sistemas lineares superiores.
    
    Entrada:
        A - matriz de coeficientes
        b - vetor de termos independentes
    Saída:
        x - vetor de incógnitas.
    '''
    n = b.shape[0]
    x = np.empty(b.shape)
    x[-1] = b[-1] / A[-1,-1]
    for i in range(n-2,-1,-1):
        soma = 0
        for j in range(i+1,n):
            soma = soma + A[i,j]*x[j]
        x[i] = (b[i] - soma) / A[i,i]
    return x

def decomp_lu(A):
    '''
    Decompor uma matriz quadrada A como um produto de duas matrizes
    triangulares.
    
        A = LU
    
    Entrada:
        A - matriz quadrada a ser decomposta.
    
    Saída:
        L - matriz triangular inferior.
        U - matriz triangular superior.
    '''
    assert len(A.shape) == 2, 'Por favor entre com uma matriz (2D)'
    assert A.shape[0] == A.shape[1], 'A matriz deve ser quadrada.'
    
    # ordem de A
    n = A.shape[0]
    
    # inicialização
    L = np.eye(n)
    U = np.zeros(A.shape)
    
    # loop principal
    for i in range(n):
        for j in range(n):
            if i <= j:
                soma = 0
                for k in range(i):
                    soma += L[i,k] * U[k,j]
                U[i,j] = A[i,j] - soma
            else:
                soma = 0
                for k in range(j):
                    soma += L[i,k] * U[k,j]
                L[i,j] = (A[i,j] - soma) / U[j,j]    
    return L, U

def lsolve_lu(A, b):
    '''
    Resolução de sistemas lineares usando decomposição LU.
    
    Entrada:
        A - matriz de coefficientes.
        b - vetor de termos independentes.
    Saída:
        x - vetor de incógnitas.
    '''
    L, U = decomp_lu(A)
    y = lsolve_triang_inf(L,b)
    x = lsolve_triang_sup(U,y)
    return x

def gauss_simples(A,b):
    '''
    Eliminação de Gauss para sistemas lineares.
    
    Entrada:
        A : matriz de coeficientes.
        b : vetor de termos independentes.
    Saída:
        A : matriz de coeficientes modificada.
        b : vetor de termos independentes modificado.
    '''
    assert len( A.shape ) == 2, 'Deve ser uma matriz.'
    assert A.shape[0] == A.shape[1], 'Deve ser matriz quadrada.'
    assert A.shape[0] == b.shape[0], 'Vetor b deve ter o mesmo número de linhas que A'
    
    n = A.shape[0]
    
    for k in range(n):
        for i in range(k+1,n):
            coeff = A[i,k] / A[k,k]
            for j in range(k,n):
                A[i,j] = A[i,j] - A[k,j] * coeff
            b[i] = b[i] - b[k] * coeff

    return A, b

def gauss_simples_mat(A,b):
    '''
    Eliminação de Gauss para sistemas lineares. Realiza operações nas linhas
    e evita problemas de pivô igual a zero.
    
    Entrada:
        A : matriz de coeficientes.
        b : vetor de termos independentes.
    Saída:
        A : matriz de coeficientes modificada.
        b : vetor de termos independentes modificado.
    '''
    assert len( A.shape ) == 2, 'A deve ser uma matriz (2D).'
    assert A.shape[0] == A.shape[1], 'Deve ser matriz quadrada.'
    assert A.shape[0] == b.shape[0], 'Vetor b deve ter o mesmo número de linhas que A'
    
    # ordem da matriz
    n = A.shape[0]

    # matriz aumentada
    Ag = np.concatenate((A, b), axis=1)
        
    for k in range(n):
        for i in range(k+1,n):
            l = 0
            while Ag[k,k] == 0:
                aux = np.array(Ag[k,:])
                Ag[k,:] = Ag[i+l,:]
                Ag[i+l,:] = aux
                l = l + 1
            coeff = Ag[i,k] / Ag[k,k]
            Ag[i] = Ag[i] - Ag[k] * coeff
    A = Ag[:,:n]
    b = Ag[:,-1].reshape(-1,1)

    return A, b

def lsolve_gauss_simples(A,b):
    '''
    Resolução de sistemas lineares usando
    eliminação de Gauss simples.
    
    Entrada:
        A - matriz de coefficientes.
        b - vetor de termos independentes.
    Saída:
        x - vetor de incógnitas.
    '''
    A, b = gauss_simples_mat(A,b)
    x = lsolve_triang_sup(A,b)
    return x

def cholesky_decomp(A):
    '''
    Decomposição de Cholesky.
    
    Entrada:
        A : matriz quadrática e simétrica.
    Saída:
        G : matriz triangular inferior.
    '''
    assert len( A.shape ) == 2, 'A deve ser uma matriz (2D).'
    assert (A.T == A).all(), 'Deve ser matriz simétrica.'
    
    # ordem da matriz
    n = A.shape[0]

    # inicialização de G
    G = np.zeros(A.shape)
    
    # decomposição de cholesky
    G[0,0] = np.sqrt(A[0,0])
    for i in range(1,n):
        G[i,0] = A[i,0] / G[0,0]
        for j in range(1, i+1):
            # elementos fora da diagonal principal
            if j < i:
                soma = 0
                for k in range(j):
                    soma += G[i,k]*G[j,k]
                G[i,j] = ( A[i,j] - soma ) / G[j,j]
            # elementos na diagonal principal
            else:
                soma = 0
                for k in range(i):
                    soma += G[i,k]*G[i,k]
                G[i,i] = np.sqrt( A[i,i] - soma )
    
    # retorna G
    return G

def lsolve_cholesky(A, b):
    '''
    Resolução de sistemas lineares usando decomposição de Cholesky.
    
    Entrada:
        A - matriz de coefficientes.
        b - vetor de termos independentes.
    Saída:
        x - vetor de incógnitas.
    '''
    G = cholesky_decomp(A)
    y = lsolve_triang_inf(G,b)
    x = lsolve_triang_sup(G.T,y)
    return x

def gauss_pivotamento_parcial(A,b):
    '''
    Eliminação de Gauss com pivotamento parcial para sistemas lineares.
    
    Entrada:
        A : matriz de coeficientes.
        b : vetor de termos independentes.
    Saída:
        A : matriz de coeficientes modificada.
        b : vetor de termos independentes modificado.
    '''
    assert len( A.shape ) == 2, 'A deve ser uma matriz (2D).'
    assert A.shape[0] == A.shape[1], 'Deve ser matriz quadrada.'
    assert A.shape[0] == b.shape[0], 'Vetor b deve ter o mesmo número de linhas que A'
    
    # ordem da matriz
    n = A.shape[0]

    # matriz aumentada
    Ag = np.concatenate((A, b), axis=1)
        
    for k in range(n):
        # ordena a matriz aumentada a partir da linha k
        ordered_indices = np.argsort(np.abs(Ag[k:,k]))[::-1]
        Ag[k:] = Ag[ordered_indices+k]
        # eliminação
        for i in range(k+1,n):
            coeff = Ag[i,k] / Ag[k,k]
            Ag[i] = Ag[i] - Ag[k] * coeff
    A = Ag[:,:n]
    b = Ag[:,-1]

    return A, b    

# testando as funções acima

# usando inversão de matrizes

In [3]:
A = np.array( [ [2., -3., 1.], [4., -6., -1.], [1., 2., 1.] ] )
print(f'A minha matriz A é\n {A}\n')
print(f'O formato de A é\n {A.shape}\n')
b = np.array( [ [-5.], [-7.], [4.] ] )
print(f'A meu vetor b é\n {b}\n')
print(f'O formato de b é\n {b.shape}\n')

A minha matriz A é
 [[ 2. -3.  1.]
 [ 4. -6. -1.]
 [ 1.  2.  1.]]

O formato de A é
 (3, 3)

A meu vetor b é
 [[-5.]
 [-7.]
 [ 4.]]

O formato de b é
 (3, 1)



In [4]:
# solução do sistema usando inversão de matrizes
x = lsolve(A, b)
print(f'O meu vetor de incógnitas é \n{x}\n')

O meu vetor de incógnitas é 
[[ 1.]
 [ 2.]
 [-1.]]



# sistemas trinagulares

In [5]:
# sistema triangular inferior
A = np.array( [ [1., 0., 0.], [3./5., 1., 0], [1/5., -3., 1.] ] )
b = np.array( [ [0.], [-7.], [-5.] ] )
print(f'A matriz A é \n {A}\n')
print(f'O vetor b é \n {b}\n')

A matriz A é 
 [[ 1.   0.   0. ]
 [ 0.6  1.   0. ]
 [ 0.2 -3.   1. ]]

O vetor b é 
 [[ 0.]
 [-7.]
 [-5.]]



In [6]:
# resolve sistema triangular
x = lsolve_triang_inf(A, b)
print(f'O  vetor x é\n {x}')

O  vetor x é
 [[  0.]
 [ -7.]
 [-26.]]


In [7]:
# sistema triangular superior
A = np.array( [ [5., 2., 1.], [0., -1./5., 17./5.], [0., 0., 13.] ] )
b = np.array( [ [0.], [-7.], [-26.] ] )
print(f'A matriz A é \n {A}\n')
print(f'O vetor b é \n {b}\n')

A matriz A é 
 [[ 5.   2.   1. ]
 [ 0.  -0.2  3.4]
 [ 0.   0.  13. ]]

O vetor b é 
 [[  0.]
 [ -7.]
 [-26.]]



In [8]:
# resolve sistema triangular
x = lsolve_triang_sup(A, b)
print(f'O  vetor x é\n {x}')

O  vetor x é
 [[-3.55271368e-16]
 [ 1.00000000e+00]
 [-2.00000000e+00]]


# decomposição LU

In [9]:
# entra com sistema linear
A = np.float32([ [5., 2., 1.], [3., 1., 4.], [1., 1., 3.] ])
b = np.float32([ [0.], [-7.], [-5.] ])
print(f'A matriz A é\n{A}')
print(f'O vetor b é\n{b}')

A matriz A é
[[5. 2. 1.]
 [3. 1. 4.]
 [1. 1. 3.]]
O vetor b é
[[ 0.]
 [-7.]
 [-5.]]


In [10]:
# decomposição LU de A
L, U = decomp_lu(A)
print(f'A matriz L é\n{L}\n')
print(f'A matriz U é\n{U}\n')
print(f'O resultado de L*U é\n {L @ U}\n')

A matriz L é
[[ 1.   0.   0. ]
 [ 0.6  1.   0. ]
 [ 0.2 -3.   1. ]]

A matriz U é
[[ 5.   2.   1. ]
 [ 0.  -0.2  3.4]
 [ 0.   0.  13. ]]

O resultado de L*U é
 [[5. 2. 1.]
 [3. 1. 4.]
 [1. 1. 3.]]



In [11]:
# resolve sistema linear usando LU
x = lsolve_lu(A,b)
print(f'O resultado é x=\n {x}')

O resultado é x=
 [[-4.4408921e-16]
 [ 1.0000000e+00]
 [-2.0000000e+00]]


# eliminação de Gauss

In [12]:
# entra com sistema linear
A = np.array( [ [6., 2., -1.], [2., 4., 1.], [3., 2., 8.] ] )
b = np.array( [ [7.], [7.], [13.] ] )
print(f'A matriz A=\n{A}\n')
print(f'O vetor b=\n{b}\n')

A matriz A=
[[ 6.  2. -1.]
 [ 2.  4.  1.]
 [ 3.  2.  8.]]

O vetor b=
[[ 7.]
 [ 7.]
 [13.]]



In [13]:
# realiza eliminação de Gauss simples
Am,bm = gauss_simples_mat(A,b)
print(f'A matriz Am=\n{Am}\n')
print(f'O vetor bm=\n{bm}\n')

A matriz Am=
[[ 6.          2.         -1.        ]
 [ 0.          3.33333333  1.33333333]
 [ 0.          0.          8.1       ]]

O vetor bm=
[[7.        ]
 [4.66666667]
 [8.1       ]]



In [14]:
# entra com sistema linear
A = np.array([ [2., -3., 1.], [4., -6., -1.], [1., 2., 1.] ])
b = np.array([ [-5.], [-7.], [4.] ])
print(f'A matriz A=\n{A}\n')
print(f'O vetor b=\n{b}\n')

A matriz A=
[[ 2. -3.  1.]
 [ 4. -6. -1.]
 [ 1.  2.  1.]]

O vetor b=
[[-5.]
 [-7.]
 [ 4.]]



In [15]:
# soluciona usando a eliminação de Gauss simples
x = lsolve_gauss_simples(A,b)
print(f'O resultado é x=\n {x}')

O resultado é x=
 [[ 1.]
 [ 2.]
 [-1.]]


# método da decomposição de Cholesky

In [16]:
# entra com sistema linear
A = np.array([ [4., 2., -4.], [2., 10., 4.], [-4., 4., 9.] ])
b = np.array([ [0.], [6.], [5.] ])
print(f'A matriz A=\n{A}\n')
print(f'O vetor b=\n{b}\n')

A matriz A=
[[ 4.  2. -4.]
 [ 2. 10.  4.]
 [-4.  4.  9.]]

O vetor b=
[[0.]
 [6.]
 [5.]]



In [17]:
# decomposição de Cholesky
G = cholesky_decomp(A)
print(f'A matriz G é\n {G}\n')

A matriz G é
 [[ 2.  0.  0.]
 [ 1.  3.  0.]
 [-2.  2.  1.]]



In [18]:
# soluciona usando a decomposição de Cholesky
x = lsolve_cholesky(A,b)
print(f'O resultado é x=\n {x}')

O resultado é x=
 [[1.]
 [0.]
 [1.]]
